In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !cp "/content/drive/MyDrive/Projet nlp/sanad_dataset.zip" /content/sanad_dataset.zip

In [3]:
# !ls -lh /content/

In [6]:
!pip install datasets
print("✔️ datasets est bien installé !")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

In [7]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import os
import pandas as pd
import numpy as np
from datasets import Dataset

In [8]:
# Vérifier si CUDA est disponible
DEVICE = "cuda" if torch.cuda.is_available() else -1
print(f"🔹 Utilisation de l'appareil : {DEVICE}")

# Charger le modèle de résumé
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50").to(DEVICE)

# Pipeline de résumé
summarization_pipeline = pipeline("summarization", model=model, tokenizer=tokenizer, device=0 if DEVICE == "cuda" else -1)


🔹 Utilisation de l'appareil : cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

Device set to use cuda:0


In [9]:
df_clean = pd.read_csv("/content/drive/MyDrive/Projet nlp/sanad_dataset_clean.csv")

In [10]:
df_clean.head()

,filename,category,text
0,4705.txt,Culture,يبدو أن أزمة الحفل الموسيقي، الذي تدخلت نقابة ...
1,2497.txt,Culture,كشف المغني البريطاني من أصل لبناني ميكا عن مشر...
2,1124.txt,Culture,أكد الفنان الكويتي عبدالحسين عبدالرضا أنه يعتب...
3,0182.txt,Culture,دشنت مدينة نابلس، أمس الاثنين، أكبر جدارية تضم...
4,0681.txt,Culture,"أقرت المغنية البريطانية ""اديل"" بأنها بكت طويلا..."


In [16]:
print(df_clean.shape)
print(f"🔹 Taille du DataFrame complet : {df_clean.shape}")

# Diviser le DataFrame en 10 chunks
chunks = np.array_split(df_clean, 1000)
print(f"🔹 Taille du premier chunk : {chunks[0].shape}")

# Travailler uniquement avec le premier chunk
first_chunk = chunks[0]
# Convertir le premier chunk en Dataset Hugging Face
dataset = Dataset.from_pandas(first_chunk)

(205760, 3)
🔹 Taille du DataFrame complet : (205760, 3)
🔹 Taille du premier chunk : (206, 3)


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [18]:
df=dataset.to_pandas()
df.head()

,filename,category,text
0,4705.txt,Culture,يبدو أن أزمة الحفل الموسيقي، الذي تدخلت نقابة ...
1,2497.txt,Culture,كشف المغني البريطاني من أصل لبناني ميكا عن مشر...
2,1124.txt,Culture,أكد الفنان الكويتي عبدالحسين عبدالرضا أنه يعتب...
3,0182.txt,Culture,دشنت مدينة نابلس، أمس الاثنين، أكبر جدارية تضم...
4,0681.txt,Culture,"أقرت المغنية البريطانية ""اديل"" بأنها بكت طويلا..."


In [19]:
from torch.utils.data import DataLoader

# Convertir le dataset Hugging Face en DataLoader pour traitement par batchs
dataloader = DataLoader(dataset["text"], batch_size=8)  # Augmenter batch_size selon la RAM disponible

# Fonction optimisée pour traiter en batch
def generate_summaries(dataloader):
    summaries = []
    for batch in dataloader:
        batch_texts = [text[:1024] for text in batch]  # Tronquer à 1024 caractères
        batch_summaries = summarization_pipeline(batch_texts, max_length=50, min_length=10, do_sample=False)
        summaries.extend([s["summary_text"] for s in batch_summaries])

    return summaries

# Générer les résumés en batch
df["summary"] = generate_summaries(dataloader)

# Sauvegarder le DataFrame
df.to_csv("/content/drive/MyDrive/Projet nlp/sanad_dataset_summarized_chunk_1.csv", index=False)
print("✔️ Résumés du premier chunk sauvegardés dans : /content/drive/MyDrive/Projet nlp/sanad_dataset_summarized_chunk_1.csv")


✔️ Résumés du premier chunk sauvegardés dans : /content/sanad_dataset_summarized_chunk_1.csv


In [20]:
# Générer les résumés en batch
df["summary"] = generate_summaries(dataloader)

# Sauvegarder le DataFrame
df.to_csv("/content/drive/MyDrive/Projet nlp/sanad_dataset_summarized_chunk_1.csv", index=False)
print("✔️ Résumés du premier chunk sauvegardés dans : /content/drive/MyDrive/Projet nlp/sanad_dataset_summarized_chunk_1.csv")

✔️ Résumés du premier chunk sauvegardés dans : /content/drive/MyDrive/Projet nlp/sanad_dataset_summarized_chunk_1.csv


In [21]:
df_summarized = pd.read_csv("/content/sanad_dataset_summarized_chunk_1.csv")

In [24]:
df_summarized['summary'][0],df_summarized['text'][0]

('يبدو أن أزمة الحفل الموسيقي، الذي تدخلت نقابة الموسيقيين في مصر لمنعه بحجة أنه من نوعية حفلات عبدة الشيطان، مازالت مستمرة، وذلك بعدما خرجت أنباء عن است',
 'يبدو أن أزمة الحفل الموسيقي، الذي تدخلت نقابة الموسيقيين في مصر لمنعه بحجة أنه من نوعية حفلات عبدة الشيطان، مازالت مستمرة، وذلك بعدما خرجت أنباء عن استقالة نقيب الموسيقيين هاني شاكر من منصبه على خلفية ما تعرض له من هجوم خلال الفترة الماضية. وفي تصريحات خاصة لـ "العربية.نت" كشف الدكتور رضا رجب وكيل نقابة الموسيقيين، أن هاني شاكر أبلغهم شفهيا باستقالته من منصبه بسبب ما تعرض له من هجوم وتطاول من بعض الأشخاص، إلا أنه لم يتقدم باستقالة مكتوبة إلى المجلس. وأشار إلى أن المجلس عقد جلسة طارئة يوم الخميس نتيجة ما حدث، وأعلن خلالها رفضه بالإجماع أي تطاول أو تجاوز يصدر تجاه "أمير الغناء العربي" هاني شاكر، كما أنه أصدر بيانا يرفض هذه الممارسات. وأكد رجب أن هاني شاكر قامة كبيرة للغاية ولديه العديد من الإنجازات التي حققها في النقابة خلال الفترة الماضية، وبالتالي لا يجوز أن يكون هناك تجاوز وتطاول على شخص بقيمته الفنية. إلى ذلك، قرر مجلس نقابة الم